In [211]:
import requests
import pandas as pd
import my_key as key
import csv
import json
import plotly.express as px

In [212]:
api_key = key.api_key

In [213]:
# Changer le chemin du fichier si nécessaire

df_gps_data = pd.read_csv("gps_data.csv")
df_gps_data.head()


,City,Latitude,Longitude
0,Mont Saint Michel,48.635954,-1.511460
1,Saint Malo,48.649518,-2.026041
2,Bayeux,49.276462,-0.702474
3,Le Havre,49.493898,0.107973
4,Rouen,49.440459,1.093966


In [214]:
# Get weather, temperature and weather
def get_temperature_humidity_weather(latitude, longitude):
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={latitude}&lon={longitude}&appid={api_key}"
    response = requests.get(url)
    data = response.json()
    if "current" in data:
        temperature = data["current"]["temp"] - 273.15  # Round temperature to the nearest whole number
        humidity = data["current"]["humidity"]
        weather = data["current"]["weather"][0]["main"]
        return temperature, humidity, weather
    else:
        return None, None, None

In [215]:
csv_filename = "meteo_data.csv"

# Créer un fichier CSV
with open('meteo_data.csv', 'w', newline='') as csvfile:
    fieldnames = ['City', 'Temperature (°C)', 'Humidity (%)', 'Weather']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Parcourir les lignes du DataFrame
    for index, row in df_gps_data.iterrows():
        city = row['City']
        latitude = row['Latitude']
        longitude = row['Longitude']
        temperature, humidity, weather = get_temperature_humidity_weather(latitude, longitude)
        if temperature is not None:
            temperature = round(temperature)  # Arrondir la température à l'entier le plus proche
            writer.writerow({'City': city, 'Temperature (°C)': temperature, 'Humidity (%)': humidity, 'Weather': weather})
            print(f"City: {city}, Temperature : {temperature}°C, Humidity : {humidity}%, Weather: {weather}")
        else:
            writer.writerow({'City': city, 'Temperature (°C)': 'N/A', 'Humidity (%)': 'N/A', 'Weather': 'N/A'})

print("Les résultats ont été enregistrés dans 'meteo_data.csv'")

City: Mont Saint Michel, Temperature : 12°C, Humidity : 96%, Weather: Clouds
City: Saint Malo, Temperature : 12°C, Humidity : 100%, Weather: Clear
City: Bayeux, Temperature : 13°C, Humidity : 84%, Weather: Clouds
City: Le Havre, Temperature : 17°C, Humidity : 94%, Weather: Clouds
City: Rouen, Temperature : 13°C, Humidity : 92%, Weather: Clouds
City: Paris, Temperature : 14°C, Humidity : 92%, Weather: Clear
City: Amiens, Temperature : 13°C, Humidity : 79%, Weather: Clear
City: Lille, Temperature : 14°C, Humidity : 84%, Weather: Clouds
City: Strasbourg, Temperature : 17°C, Humidity : 85%, Weather: Clouds
City: Chateau du Haut Koenigsbourg, Temperature : 15°C, Humidity : 82%, Weather: Clear
City: Colmar, Temperature : 18°C, Humidity : 86%, Weather: Clear
City: Eguisheim, Temperature : 18°C, Humidity : 86%, Weather: Clear
City: Besancon, Temperature : 15°C, Humidity : 96%, Weather: Clouds
City: Dijon, Temperature : 16°C, Humidity : 77%, Weather: Clouds
City: Annecy, Temperature : 18°C, Hum

In [216]:
# Changer le chemin du fichier si nécessaire

df_meteo = pd.read_csv("meteo_data.csv")
df_meteo.head()

,City,Temperature (°C),Humidity (%),Weather
0,Mont Saint Michel,12,96,Clouds
1,Saint Malo,12,100,Clear
2,Bayeux,13,84,Clouds
3,Le Havre,17,94,Clouds
4,Rouen,13,92,Clouds


In [217]:
df_meteo.rename(columns={'temperature (°C)': 'temperature', 'humidite (%)': 'humidite'}, inplace=True)
df_meteo.head()

,City,Temperature (°C),Humidity (%),Weather
0,Mont Saint Michel,12,96,Clouds
1,Saint Malo,12,100,Clear
2,Bayeux,13,84,Clouds
3,Le Havre,17,94,Clouds
4,Rouen,13,92,Clouds


Création d'un seul fichier csv avec toute les données

In [218]:
with open('booking.json') as f:
    data_json = json.load(f)

In [219]:
# Modifier les noms de ville dans le JSON
for entry in data_json:
    if 'ville' in entry:
        entry['ville'] = entry['ville'].replace('+', ' ')

In [220]:
# Créer un DataFrame à partir du JSON
df_json = pd.DataFrame(data_json)
df_json.head()

,ville,name,website,score,desc,lat,lon
0,Aigues-Mortes%2C Languedoc-Roussillon%2C France,"appartement n2 centre village, clim, parking g...",https://www.booking.com/hotel/fr/appartement-n...,3 sur 5,Hébergement géré par un particulier,43.56724070,4.19380070
1,Aigues-Mortes%2C Languedoc-Roussillon%2C France,Cabane STELLA Aigues-Mortes,https://www.booking.com/hotel/fr/la-cabane-du-...,3 sur 5,Hébergement géré par un particulier,43.57930210,4.21283370
2,Aigues-Mortes%2C Languedoc-Roussillon%2C France,LE FLAMANT ROSE - COSYKAZ,https://www.booking.com/hotel/fr/le-flamant-ro...,3 sur 5,Hébergement géré par un particulier,43.57152370,4.17889910
3,Aigues-Mortes%2C Languedoc-Roussillon%2C France,B&B La Terre Brûlée,https://www.booking.com/hotel/fr/b-amp-b-la-te...,3 sur 5,"Situé à Aigues-Mortes, le B&B La Terre Brûlée ...",43.56197612,4.20767969
4,Aigues-Mortes%2C Languedoc-Roussillon%2C France,Au Cœur des Remparts,https://www.booking.com/hotel/fr/au-coeur-des-...,3 sur 5,Hébergement géré par un particulier,43.56540100,4.19297300


In [221]:
df_json.rename(columns={"ville" : "City"}, inplace=True)

In [222]:
df_fusionne = df_meteo.merge(df_json, on='City', how='left')
df_fusionne.head()

,City,Temperature (°C),Humidity (%),Weather,name,website,score,desc,lat,lon
0,Mont Saint Michel,12,96,Clouds,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,2 sur 5,L’Hotel Vert vous propose des chambres décorée...,48.61470049,-1.50961697
1,Mont Saint Michel,12,96,Clouds,La Greve Saint Michel,https://www.booking.com/hotel/fr/la-greve-sain...,3 sur 5,Hébergement géré par un particulier,48.59775819,-1.51228422
2,Mont Saint Michel,12,96,Clouds,Maison au pied du Mont Saint Michel 2,https://www.booking.com/hotel/fr/maison-au-pie...,3 sur 5,Hébergement géré par un particulier,48.61561170,-1.48851210
3,Mont Saint Michel,12,96,Clouds,Gîte Caribou,https://www.booking.com/hotel/fr/gite-caribou....,3 sur 5,Hébergement géré par un particulier,48.62259438,-1.39050267
4,Mont Saint Michel,12,96,Clouds,Les Hortensias,https://www.booking.com/hotel/fr/les-hortensia...,3 sur 5,Hébergement géré par un particulier,48.59456300,-1.61772500


In [223]:
new_col_names = {
    'City': 'city',
    'Temperature (°C)': 'temperature',
    'Humidity (%)': 'humidity',
    'Weather': 'weather',
    'name': 'hotel',
    'website': 'website',
    'score': 'score',
    'desc': 'description',
    'lat': 'lat',
    'lon': 'lon',
}


In [224]:
df_fusionne = df_fusionne.rename(columns=new_col_names)
df_fusionne.head()

,city,temperature,humidity,weather,hotel,website,score,description,lat,lon
0,Mont Saint Michel,12,96,Clouds,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,2 sur 5,L’Hotel Vert vous propose des chambres décorée...,48.61470049,-1.50961697
1,Mont Saint Michel,12,96,Clouds,La Greve Saint Michel,https://www.booking.com/hotel/fr/la-greve-sain...,3 sur 5,Hébergement géré par un particulier,48.59775819,-1.51228422
2,Mont Saint Michel,12,96,Clouds,Maison au pied du Mont Saint Michel 2,https://www.booking.com/hotel/fr/maison-au-pie...,3 sur 5,Hébergement géré par un particulier,48.61561170,-1.48851210
3,Mont Saint Michel,12,96,Clouds,Gîte Caribou,https://www.booking.com/hotel/fr/gite-caribou....,3 sur 5,Hébergement géré par un particulier,48.62259438,-1.39050267
4,Mont Saint Michel,12,96,Clouds,Les Hortensias,https://www.booking.com/hotel/fr/les-hortensia...,3 sur 5,Hébergement géré par un particulier,48.59456300,-1.61772500


In [225]:
df_final = df_fusionne[
    ['city', 'temperature', 'humidity', 'weather', 
     'hotel','website','score','description','lat','lon'
     ]
    ]


In [226]:
df_final.to_csv('data_final.csv', index=True, index_label='id')


In [227]:
# Changer le chemin du fichier si nécessaire

df_fichier_final = pd.read_csv("data_final.csv")
df_fichier_final.head()

,id,city,temperature,humidity,weather,hotel,website,score,description,lat,lon
0,0,Mont Saint Michel,12,96,Clouds,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,2 sur 5,L’Hotel Vert vous propose des chambres décorée...,48.614700,-1.509617
1,1,Mont Saint Michel,12,96,Clouds,La Greve Saint Michel,https://www.booking.com/hotel/fr/la-greve-sain...,3 sur 5,Hébergement géré par un particulier,48.597758,-1.512284
2,2,Mont Saint Michel,12,96,Clouds,Maison au pied du Mont Saint Michel 2,https://www.booking.com/hotel/fr/maison-au-pie...,3 sur 5,Hébergement géré par un particulier,48.615612,-1.488512
3,3,Mont Saint Michel,12,96,Clouds,Gîte Caribou,https://www.booking.com/hotel/fr/gite-caribou....,3 sur 5,Hébergement géré par un particulier,48.622594,-1.390503
4,4,Mont Saint Michel,12,96,Clouds,Les Hortensias,https://www.booking.com/hotel/fr/les-hortensia...,3 sur 5,Hébergement géré par un particulier,48.594563,-1.617725


In [228]:
df_meteo.head()

,City,Temperature (°C),Humidity (%),Weather
0,Mont Saint Michel,12,96,Clouds
1,Saint Malo,12,100,Clear
2,Bayeux,13,84,Clouds
3,Le Havre,17,94,Clouds
4,Rouen,13,92,Clouds


Top 5 weather localization

In [229]:
filtered_df = df_fichier_final[(df_fichier_final["weather"] == "Clear") | (df_fichier_final["weather"] == "light rain")]
filtered_df

,id,city,temperature,humidity,weather,hotel,website,score,description,lat,lon
20,20,Saint Malo,12,100,Clear,NaN,NaN,NaN,NaN,NaN,NaN
81,81,Paris,14,92,Clear,District Nation,https://www.booking.com/hotel/fr/alfa-paris-na...,2 sur 5,"Situé à Paris, à 500 mètres de la place de la ...",48.851311,2.392413
82,82,Paris,14,92,Clear,Hôtel De Venise,https://www.booking.com/hotel/fr/de-venise.fr....,2 sur 5,Situé à proximité de la Bastille et de la Gare...,48.845272,2.383612
83,83,Paris,14,92,Clear,La Maison Montparnasse,https://www.booking.com/hotel/fr/la-maison-mon...,2 sur 5,La Maison Montparnasse est située à 10 minutes...,48.832519,2.316845
84,84,Paris,14,92,Clear,Grand Hotel des Balcons,https://www.booking.com/hotel/fr/grand-des-bal...,2 sur 5,Situé dans le quartier chic de Saint-Germain à...,48.850422,2.339299
...,...,...,...,...,...,...,...,...,...,...,...
657,657,La Rochelle,15,94,Clear,Hôtel Le Bord'O Vieux Port,https://www.booking.com/hotel/fr/hoteldebordea...,2 sur 5,"Occupant un bâtiment du XVIIIe siècle, le Hôte...",46.156877,-1.149112
658,658,La Rochelle,15,94,Clear,Accostage Hôtel Plage de la Concurrence,https://www.booking.com/hotel/fr/accostage.fr....,2 sur 5,L'Accostage Hôtel Plage de la Concurrence est ...,46.157124,-1.166042
659,659,La Rochelle,15,94,Clear,Hôtel de l'Océan,https://www.booking.com/hotel/fr/de-l-ocean-la...,2 sur 5,"Construit en 1646, l'Hôtel de l'Océan se trouv...",46.157020,-1.154128
660,660,La Rochelle,15,94,Clear,"Hotel La Marine, Vieux Port",https://www.booking.com/hotel/fr/la-marine-la-...,2 sur 5,L'Hôtel La Marine est situé dans le centre-vil...,46.158159,-1.152671


In [249]:
# Best weather
filtered_df = df_fichier_final[(df_fichier_final["weather"] == "Clear") | (df_fichier_final["weather"] == "light rain")]

# Then group by 'city', select the first entry, sort by 'temperature', and take the top 5
df_top_five = filtered_df.groupby(by='city').first().sort_values(by=['temperature'], ascending=False).reset_index().head(10)

df_top_five.head()

,city,id,temperature,humidity,weather,hotel,website,score,description,lat,lon
0,Bormes les Mimosas,341,23,53,Clear,Hotel des Arts - Cite Bergere,https://www.booking.com/hotel/fr/des-arts-pari...,2 sur 5,L'Hotel des Arts est situé dans le centre de P...,48.871880,2.344655
1,Aigues Mortes,481,22,53,Clear,None,None,None,None,NaN,NaN
2,Saintes Maries de la mer,482,22,57,Clear,Hotel Mas des Lys,https://www.booking.com/hotel/fr/mas-des-lys-l...,3 sur 5,L'Hôtel Mas des Lys est situé à Saintes-Maries...,43.474188,4.410206
3,Marseille,381,22,59,Clear,Love room Jacuzzi Sauna Privatif Marseille,https://www.booking.com/hotel/fr/loft-love-roo...,3 sur 5,Hébergement géré par un particulier,43.296098,5.487121
4,Collioure,502,21,55,Clear,Grand Hotel des Balcons,https://www.booking.com/hotel/fr/grand-des-bal...,2 sur 5,Situé dans le quartier chic de Saint-Germain à...,48.850422,2.339299


In [250]:
csv_filename = "top_five_weather.csv"

# Créer un fichier CSV
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ['City', 'Temperature (°C)', 'Weather']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Parcourir les lignes du DataFrame
    for index, row in df_top_five.iterrows():
        city = row['city']
        temperature = row['temperature']
        weather = row['weather']
        writer.writerow({'City': city, 'Temperature (°C)': temperature, 'Weather': weather})
        print(f"City: {city}, Temperature : {temperature}°C, Weather: {weather}")
   

print("Les résultats ont été enregistrés dans 'meteo_data.csv'")

City: Bormes les Mimosas, Temperature : 23°C, Weather: Clear
City: Aigues Mortes, Temperature : 22°C, Weather: Clear
City: Saintes Maries de la mer, Temperature : 22°C, Weather: Clear
City: Marseille, Temperature : 22°C, Weather: Clear
City: Collioure, Temperature : 21°C, Weather: Clear
City: Avignon, Temperature : 21°C, Weather: Clear
City: Nimes, Temperature : 21°C, Weather: Clear
City: Uzes, Temperature : 20°C, Weather: Clear
City: Colmar, Temperature : 18°C, Weather: Clear
City: Eguisheim, Temperature : 18°C, Weather: Clear
Les résultats ont été enregistrés dans 'meteo_data.csv'


Top 5 map

In [301]:
df_top_five.head()

,city,id,temperature,humidity,weather,hotel,website,score,description,lat,lon
0,Bormes les Mimosas,341,23,53,Clear,Hotel des Arts - Cite Bergere,https://www.booking.com/hotel/fr/des-arts-pari...,2 sur 5,L'Hotel des Arts est situé dans le centre de P...,48.871880,2.344655
1,Aigues Mortes,481,22,53,Clear,None,None,None,None,NaN,NaN
2,Saintes Maries de la mer,482,22,57,Clear,Hotel Mas des Lys,https://www.booking.com/hotel/fr/mas-des-lys-l...,3 sur 5,L'Hôtel Mas des Lys est situé à Saintes-Maries...,43.474188,4.410206
3,Marseille,381,22,59,Clear,Love room Jacuzzi Sauna Privatif Marseille,https://www.booking.com/hotel/fr/loft-love-roo...,3 sur 5,Hébergement géré par un particulier,43.296098,5.487121
4,Collioure,502,21,55,Clear,Grand Hotel des Balcons,https://www.booking.com/hotel/fr/grand-des-bal...,2 sur 5,Situé dans le quartier chic de Saint-Germain à...,48.850422,2.339299


In [355]:
colorscale = [
[0, 'rgb(57, 162, 225)'],
[1, 'rgb(234, 32, 41)']
]
fig = px.scatter_mapbox(df_top_five, lat="lat", lon="lon", color="temperature", size="humidity",
                         mapbox_style="open-street-map", zoom = 4, color_continuous_scale=colorscale, height=510, width=700)
fig.show()

Top 20 map

In [293]:
pd.set_option('display.max_rows', None)

In [339]:
df_top_20 =  df_final[df_final['city'].isin(df_top_five.loc[:,'city'].to_list())].loc[1:,['city', 'hotel', 'score', 'lat', 'lon', 'temperature']]
df_top_20.head()

,city,hotel,score,lat,lon,temperature
181,Colmar,Greet Hotel Colmar,3 sur 5,48.06517112,7.36302316,18
182,Colmar,Le grand gîte du Sorcier ! Logement magique,3 sur 5,48.06972050,7.35878970,18
183,Colmar,Aux Bijoux de Colmar - Le Saphir - 1 PARKING G...,3 sur 5,48.07733814,7.35609418,18
184,Colmar,Hôtel Turenne,3 sur 5,48.07192026,7.36053944,18
185,Colmar,"Bel appartement neuf garage, parking et terrasse",3 sur 5,48.06907980,7.37845130,18


In [340]:
df_top_20['score'] = df_top_20['score'].astype(str).str[0]
df_top_20['score'] = pd.to_numeric(df_top_20['score'], errors='coerce')
df_top_20.head()


,city,hotel,score,lat,lon,temperature
181,Colmar,Greet Hotel Colmar,3.0,48.06517112,7.36302316,18
182,Colmar,Le grand gîte du Sorcier ! Logement magique,3.0,48.06972050,7.35878970,18
183,Colmar,Aux Bijoux de Colmar - Le Saphir - 1 PARKING G...,3.0,48.07733814,7.35609418,18
184,Colmar,Hôtel Turenne,3.0,48.07192026,7.36053944,18
185,Colmar,"Bel appartement neuf garage, parking et terrasse",3.0,48.06907980,7.37845130,18


In [353]:
df_top_20['lat'] = pd.to_numeric(df_top_20['lat'], errors='coerce')
df_top_20['lon'] = pd.to_numeric(df_top_20['lon'], errors='coerce')

In [367]:
fig = px.scatter_mapbox(df_top_20, lat="lat", lon="lon", color="score", mapbox_style="open-street-map", zoom = 10, size = "temperature", color_continuous_scale="oranges", 
                        hover_data={"city": True, "hotel": True, "lat": False, "lon": False}, height=510, width=700)
fig.show()